In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
# import openpyxl
import time, os, requests, threading, ast

In [2]:
def scrape(path):
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)

# =====================================================================================================
    # complete login
# =====================================================================================================
    # provide username 
    try:
        provided_username = "fourbrotherstrading@icloud.com"
        username_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "loginForm.Email")))
        username_field.send_keys(provided_username)
    except Exception as e:
        print(f"No email button found and the error is--> {e}")
        
    # get the password id from website
    try:
        provided_password = "Muhssan7865"
        pass_field = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "loginForm.Password"))) #driver.find_element(By.ID, "loginForm.Password")
        pass_field.send_keys(provided_password)
    except Exception as e:
        print(f"No password tab found and the error is--> {e}")

    # get the login button name-value
    try:
        login_button = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, "//input[@value = 'Login']"))) #driver.find_element(By.XPATH, "//input[@value = 'Login']")
        login_button.click()
    except Exception as e:
        print(f"No login button found and error is --> {e}")
# =====================================================================================================
# =====================================================================================================   

    results = []
    car_count = 0
    while True:
        if not driver.window_handles:
            print("Browser window closed. Stopping script.")
            break
        
        details = {}
        
        try:
            title = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/h1[@class="vehicle-details__title"]'))) 
            if title:
                details['Title']  = title.text.strip()
            else:
                details['Title'] = 'na'
        except:
            print("No title tab found")
        
        try:
            car_props = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details__properties "]')))
            if car_props:
                details['Transmission'] = car_props.text.strip().split(" - ")[0]
                details['Fuel Type'] = car_props.text.strip().split(" - ")[1]
                details['CC'] = car_props.text.strip().split(" - ")[2]
                details['Doors'] = car_props.text.strip().split(" - ")[3].split(" ")[0]
                details['Body Type'] = car_props.text.strip().split(" - ")[3].split(" ")[2]
            else:
                details['Transmission'] = 'na'
                details['Fuel Type'] = 'na'
                details['CC'] = 'na'
                details['Doors'] = 'na'
                details['Body Type'] = 'na'
        except:
            print("no car props")
        
        try:
            grade = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//a[contains(@class,"button_grade")])[1]'))).text.strip()
            if grade:
                details['Grade'] = grade.split(" ")[1]
            # else:
            #     details['Grade'] = 'na'
        except:
            details['Grade'] = 'na'
        
        try:
            cp = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//a[contains(@title,"CheckPoint")])[1]'))).text.strip()
            if cp:
                details['cp'] = cp
            # else:
            #     details['cp'] = 'na'
        except:
            details['cp'] = 'na'
        
        try:
            sc = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//a[contains(@title,"SureCheck")])[1]'))).text.strip()
            if sc:
                details['sc'] = sc
            # else:
            #     details['sc'] = 'na'
        except:
            details['sc'] = 'na'
                    
        try:
            img_main_card =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="flexslider flexslider__vehicle js-vehicle-image"]')))  
            if img_main_card:
                imgs_ul =WebDriverWait(img_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="slides"]')))
                if imgs_ul:
                    images = []
                    imgs_lis = WebDriverWait(imgs_ul, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, "li")))
                    if imgs_lis:
                        for img in imgs_lis:
                            images.append(img.get_attribute("data-thumb"))

                    img_str = ", ".join(images)
                    details["Images"] =  img_str
        except Exception as e:
            print("No images found")
                        
        try: 
            veh_info_card =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="vdb__content js-dropdown-container"])[1]')))   
            if veh_info_card:
                car_info_items = veh_info_card.find_elements(By.CLASS_NAME, "vdb__item")
                # inside each info_item
                for info_item in car_info_items:
                    # get label
                    info_label = info_item.find_element(By.CLASS_NAME, "vdb__label").text.strip()
                    info_value = info_item.find_element(By.CLASS_NAME, "vdb__value").text.strip()
                    if info_label and info_value:
                        # get value
                        if info_label.lower() == "odometer":
                            details['Mileage']  = info_value
                        else:
                            details[info_label] = info_value if info_value else "na"
            else:
                print("No veh info card found")
        except Exception as e:
            print("No veh details")
        
        try: 
            Private_Hire =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@id="divProvenenceCheckSummary"]/div/div/div)[1]/span'))).text.strip()
            if Private_Hire:
                details['Private Hire'] = Private_Hire
            # else:
            #     details['Private Hire'] = "na"
        except:
            details['Private Hire'] = "na"
         
        try:   
            MOT_Mileage_Discrepancy =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@id="divProvenenceCheckSummary"]/div/div/div)[2]/span'))).text.strip()
            if MOT_Mileage_Discrepancy:
                details['MOT Mileage Discrepancy'] = MOT_Mileage_Discrepancy
            # else:
            #     details['MOT Mileage Discrepancy'] = "na"
        except:
            details['MOT Mileage Discrepancy'] = "na"
        
        try: 
            vendor =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="vehicle-details__block vdb vdb_sales"]/div/div/span/a)[3]'))).text.strip() 
            if vendor:
                details['Vendor'] = vendor
            # else:
            #     details['Vendor'] = 'na'
        except:
            details['Vendor'] = 'na'
          
        try:      
            location = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="vehicle-details__block vdb vdb_sales"]/div/div/span/a)[1]'))).text.strip()
            if location:
                details['Center'] = location.split("(")[-1]
        except:
            details['Center'] = 'na'
            
        try:
            spec_card=WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="vehicle-details__block vehicle-details__clear vdb"])[1]/div/div/ul')))  
            if spec_card:
                specs = spec_card.find_elements(By.TAG_NAME, "li")
                if specs:
                    specs_lst = [spec.text.strip() for spec in specs]
                    details['Features'] = ", ".join(specs_lst)
                else:
                    details['Features'] = "na"
        except:
            print("No specs")  
        
        try:
            add_card=WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="vehicle-details__block vehicle-details__clear vdb"])[2]/div/div/div')))  
            if add_card:
                adds = add_card.find_elements(By.TAG_NAME, "li")
                if adds:
                    adds_lst = [ad.text.strip() for ad in adds]
                    details['Additional Information'] = ", ".join(adds_lst)
                else:
                    details['Additional Information'] = "na"
            else:
                details['Additional Info'] = "na"
        except:
            print("No Additional Info") 
                    
        try:
            buy_now = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@class="current-price current-price_buy-now"]'))) 
            if buy_now:
                details['Buy Now Price'] = buy_now.text.strip()
            # else:
            #     details['Buy Now Price'] = 'na'
        except:
            details['Buy Now Price'] = 'na'
        
        try:
            event = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="column column100"]')))
            if event:
                event_name = WebDriverWait(event, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@class="bold widget-event-name"]'))) 
                if event_name:
                    details['Auction Name'] = event_name.text.strip()
                else:
                    details['Auction Name'] = 'na' 
                    
                event_date_time = WebDriverWait(event, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="startDate"]'))) 
                if event_date_time:
                    details['Start Date'] = event_date_time.text.strip().split(" ")[0]
                    details['Start Time'] = event_date_time.text.strip().split(" ")[1]
                else:
                    details['Start Date'] = 'na' 
                    details['Start Time'] = "na"
                
                lot = event.find_element(By.XPATH, '(.//div[@class = "block__column bold"])[1]').text.strip()
                if lot:
                    details['Lot'] = lot
                else:
                    details['Lot'] = "na"
        except:
            print("No event_name")    

        # get the pricing information --> done
        try:
            pricing_card=WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details__block vdb vdb_grey vdb_valuations"]')))
            if pricing_card:
                pricing_data = WebDriverWait(pricing_card, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vdb__content vdb__content_table js-dropdown-container"]')))
                if pricing_data:
                    pricing_items = WebDriverWait(pricing_data, 2).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vdb__item"]')))
                    if pricing_items:
                        for pricing_item in pricing_items:
                                pricing_labels = pricing_item.find_element(By.CLASS_NAME, "vdb__label").text.strip()
                                pricing_value = pricing_item.find_element(By.CLASS_NAME, "vdb__value")
                                if pricing_labels and pricing_value:
                                    details[pricing_labels] = pricing_value.text.strip()
                                else:
                                    details[pricing_labels] = "N/A"
                    else:
                        print("No pricing labels found")
                else:
                    print("No pricing table found")
            else:
                print("No pricing items found")
        except Exception as e:
            print(f"No pricing tab found")        
                    
        # inspection --> done
        try:
            # Wait for all blocks with the class `vehicle-details___block vdb`
            isnpec_service = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="vdb__content vdb__content_table js-dropdown-container"])[2]')))
            # print(len(vehicle_details_blocks))
            
            if isnpec_service:
                details_items = isnpec_service.find_elements(By.XPATH, './/div[@class="vdb__item"]')
                
                if details_items:
                    for item in details_items:
                        # Extract label and value for each item
                        label = item.find_element(By.XPATH, './/div[@class="vdb__label"]').text.strip()
                        value_element = item.find_element(By.XPATH, './/span[@class="vdb__value"]')
                        
                        # Handle cases where value contains links
                        if value_element.find_elements(By.TAG_NAME, 'a'):
                            value = value_element.find_element(By.TAG_NAME, 'a').get_attribute("href")
                        else:
                            value = value_element.text.strip()
                        
                        # Add the label and value to the details dictionary
                        details[label] = value
        except Exception as e:
            print(f"Error locating vehicle details blocks")
            
        try:
            service_history_card = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details__block vdb vdb_table"]/div')))
            
            if service_history_card:
                # Get headers
                headers = [header.text.strip() for header in service_history_card.find_elements(By.XPATH, './/div[@class="vdb__td vdb__th"]')]
                
                # Initialize dictionary to store data with empty lists
                for header in headers:
                    details[header] = []
                
                # Get all data rows (excluding header rows)
                rows = service_history_card.find_elements(By.XPATH, './/div[@class="vdb__row"]')
                
                for row in rows:
                    cells = row.find_elements(By.CLASS_NAME, "vdb__td")
                    # if len(cells) == len(headers):
                    for i, cell in enumerate(cells):
                        details[headers[i]].append(cell.text.strip())
                    
        except Exception as e:
            print(f"Error extracting service history:")
        
        try:
            MOT_history_card = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="vehicle-details__block vdb vdb_table"]/div)[2]')))
            
            if MOT_history_card:
                # Get headers
                mot_headers = [mot_header.text.strip() for mot_header in MOT_history_card.find_elements(By.XPATH, './/div[@class="vdb__td vdb__th"]')]
                
                # Initialize dictionary to store data with empty lists
                for mot_header in mot_headers:
                    details[mot_header] = []
                
                # Get all data rows (excluding header rows)
                mot_rows = MOT_history_card.find_elements(By.XPATH, './/div[@class="vdb__row"]')
                
                for mot_row in mot_rows:
                    mot_cells = mot_row.find_elements(By.CLASS_NAME, "vdb__td")
                    for i, mot_cell in enumerate(mot_cells):
                        details[mot_headers[i]].append(mot_cell.text.strip())
                    
        except Exception as e:
            print(f"Error extracting service history:")
        # car_count+=1
        results.append(details)
        try:
            nxt = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/a[@id="nextLink"]'))) 
            if nxt:
                nxt.click()
                time.sleep(2)
            else:
                break
        except:
            print("no next")
            break
        
    df = pd.DataFrame.from_dict(results)
    df.to_csv("manheim_data.csv", index=False)
    driver.quit()
path = "https://www.manheim.co.uk/vehicle-detail/FORD/TRANSIT?id=3951616&returnUrl=%2Fcatalogues-and-events%2Flisting%2F46426%3FPage%3D1%26PageSize%3D30%26IsKm%3DFalse%26SaleEventId%3D46426%26SortBy%3DLotNumber%26SortAscending%3DTrue%233951616&indx=2"
scrape(path)

No event_name
Error extracting service history:
Error extracting service history:
No event_name
Error extracting service history:
no next


In [ ]:
file_path = "Shotts NON-RUNNER SALE 930am.xlsx"

def load_excel_or_csv(file_path):
    """Auto-detect Excel/CSV and return DataFrame"""
    try:
        # 🔹 Try Excel read
        df = pd.read_excel(file_path, header=None, engine="openpyxl")
        print("✅ Loaded as Excel successfully.")
        return df
    except Exception as e1:
        print(f"⚠️ Excel load failed ({e1}) — trying CSV...")
        try:
            # 🔹 Try CSV read (auto detect separator)
            df = pd.read_csv(file_path, header=None, sep=None, engine="python")
            print("✅ Loaded as CSV successfully.")
            return df
        except Exception as e2:
            print(f"❌ Both Excel and CSV failed: {e2}")
            print("Creating empty DataFrame...")
            # 🔹 Create empty fallback DataFrame
            return pd.DataFrame(columns=["REG NO", "Manufacturer", "Model", "Derivative"])

# ✅ Load data safely
df = load_excel_or_csv(file_path)

# ✅ Clean + standardize
if not df.empty:
    df.columns = df.iloc[1]
    df = df[1:]
    df.reset_index(drop=True, inplace=True)
    df = df[1:]
    if "Reg No." in df.columns:
        df.rename(columns={"Reg No.": "REG NO"}, inplace=True)

    df.reset_index(drop=True, inplace=True)
    df.to_csv("to_merge.csv", index=False)
else:
    print("⚠️ Empty DataFrame created — check input file content.")
    df.to_csv("to_merge.csv", index=False)

# ✅ Continue with merge
top_car_df = pd.read_csv("to_merge.csv")
top_car_live_cleaned_df = pd.read_csv("manheim_data.csv")

merged_df = top_car_live_cleaned_df.merge(
    top_car_df[['REG NO', 'Manufacturer', 'Model', 'Derivative']],
    on='REG NO',
    how='left'
)

# ✅ Save merged data
output_path = "manheim_data_merged.csv"
merged_df.to_csv(output_path, index=False)

# ✅ Cleanup temp files
for f in ["to_merge.csv", "manheim_data.csv"]:
    if os.path.exists(f):
        os.remove(f)

print("🎉 Merge complete! File saved as:", output_path)

⚠️ Excel load failed (File is not a zip file) — trying CSV...
❌ Both Excel and CSV failed: Could not determine delimiter
Creating empty DataFrame...
⚠️ Empty DataFrame created — check input file content.
🎉 Merge complete! File saved as: manheim_data_merged.csv


In [4]:
df = pd.read_csv("manheim_data_merged.csv")
df.columns

Index(['Title', 'Transmission', 'Fuel Type', 'CC', 'Doors', 'Body Type',
       'Grade', 'cp', 'sc', 'Images', 'REG NO', 'MFR YEAR', 'REG DATE',
       'Mileage', 'BASE COLOUR', 'COLOUR', 'COLOUR TYPE', 'INTERIOR COLOUR',
       'TRIM TYPE', 'NO OF SEATS', 'NO OF KEYS', 'V5', 'VAT', 'TOTAL LOSS',
       'RUNNER', 'IMPORTED', 'Private Hire', 'MOT Mileage Discrepancy',
       'Vendor', 'Center', 'Features', 'Additional Information',
       'Buy Now Price', 'CAP CLEAN', 'CAP AVERAGE', 'CAP RETAIL', 'GRADE',
       'INSPECTION REPORT', 'REGISTERED KEEPERS', 'NO OF SERVICES',
       'DATE OF LAST SERVICE', 'ODOMETER AT LAST SERVICE', 'MOT EXPIRY DATE',
       'SERVICE DATE', 'ODOMETER', 'SERVICED BY', 'SERVICE RECORD TYPE',
       'Manufacturer', 'Model', 'Derivative'],
      dtype='object')

In [5]:
df = pd.read_csv("manheim_data_merged.csv")
df.rename(columns={"REG NO": "Reg", "MFR YEAR":"Year", "REG DATE":"D.O.R", "NO OF SEATS":"Seats", "NO OF KEYS":"Keys", "GRADE":"Grade Link", "TOTAL LOSS":"T Loss", "REGISTERED KEEPERS":"Former Keepers", "DATE OF LAST SERVICE":"Last Service", "Derivative":"Variant", "Manufacturer":"Make", "ODOMETER":"DVSA Mileage", "VAT":"VAT Status", "RUNNER":"Engine Runs", "ODOMETER AT LAST SERVICE":"Last Service mileage", "SERVICE DATE":"Service History"}, inplace=True)
df['Mileage warranted'] = df['Mileage'].apply(lambda x: x.split('\n')[1] if isinstance(x, str) and '\n' in x else None)
# df['Mileage warranted'] = df['Mileage'].apply(lambda x: x.split('\n')[1] if isinstance(x, str) and '\n' in x else None)
df['Mileage warranted'] = df['Mileage warranted'].apply(lambda x: x.split("(")[1].split(")")[0] if isinstance(x, str) and "(" in x and ")" in x else 'na')
df['Mileage'] = df['Mileage'].apply(lambda x:x.split(" ")[0] if isinstance(x, str) and " " in x else "na")
df.to_csv("manheim_data_cleaned.csv", index=False)
os.remove("manheim_data_merged.csv")

### images and Inspection report

In [6]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from Manheim Auction"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [7]:
file_path = "manheim_data_cleaned.csv"
df = pd.read_csv(file_path)

# 🔹 Ensure column exists
if "Images" in df.columns:
    # Replace ims_tiny with zoom in all URLs
    df["Images"] = df["Images"].astype(str).str.replace(
        "/ims_tiny/", "/zoom/", regex=False
    )

    # 🔹 Save updated file (overwrite)
    df.to_csv(file_path, index=False)
    print("✅ All image URLs updated successfully in:", file_path)
else:
    print("⚠️ 'Images' column not found in the file.")

✅ All image URLs updated successfully in: manheim_data_cleaned.csv


In [8]:
from urllib.parse import urlparse, urljoin


df =pd.read_csv("manheim_data_cleaned.csv")
reg_img = df[["Reg", "Images"]]
report = df[['Reg', 'INSPECTION REPORT']]

def download_reports(data, main_folder="Inspec_report"): 
    

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        report_urls = row["INSPECTION REPORT"]

        if not report_urls or pd.isna(report_urls):
            print(f"Missing Inspection Report of {reg_no}")

            
        else:
            if not report_urls.startswith(("http://", "https://")): 
                report_urls = urljoin("https://", report_urls) 
            

            parsed_url = urlparse(report_urls)
            

            try:

                response = requests.get(report_urls, stream=True) 
                response.raise_for_status() 
                

                file_name = os.path.basename(parsed_url.path) or f"inspec_repo.pdf"


                file_extension = file_name.split(".")[-1]
                

                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"inspec_repo.pdf" 
                

                full_file_name = os.path.join(main_folder, f"{reg_no}.pdf")
                

                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): 
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {report_urls} for {reg_no}: {e}")


def download_images(data, main_folder="Images"): 
    

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        if pd.isna(row["Images"]) or not isinstance(row["Images"], str) or row["Images"].strip() == "":
            print(f"Skipping {reg_no} due to missing images.")
            continue  

        image_urls = row["Images"].split(", ")
        

        reg_folder = os.path.join(main_folder, reg_no) 
        os.makedirs(reg_folder, exist_ok=True) 
        

        for idx, url in enumerate(image_urls):
            url = url.strip() 
            if not url.startswith(("http://", "https://")): 
                url = urljoin("https://", url) 
            

            parsed_url = urlparse(url)

 
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            

            try:

                response = requests.get(url, stream=True)
                response.raise_for_status() 

                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"


                file_extension = file_name.split(".")[-1]
                

                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" 
                

                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                

                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)


                add_watermark_to_image(full_file_name)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

def start_funcs():
    thread1 = threading.Thread(target=download_reports, args=(report,))
    thread2 = threading.Thread(target=download_images, args=(reg_img,))


    thread1.start()
    thread2.start()
    

    thread1.join()
    thread2.join()

if __name__ == "__main__":
    start_funcs()

Watermark added to Images\WN25UJW\WN25UJW_1.jpg
Downloaded: Images\WN25UJW\WN25UJW_1.jpg
Downloaded: Inspec_report\WN25UJW.pdf
Downloaded: Inspec_report\BP68UNZ.pdf
Watermark added to Images\WN25UJW\WN25UJW_2.jpg
Downloaded: Images\WN25UJW\WN25UJW_2.jpg
Watermark added to Images\WN25UJW\WN25UJW_3.jpg
Downloaded: Images\WN25UJW\WN25UJW_3.jpg
Watermark added to Images\WN25UJW\WN25UJW_4.jpg
Downloaded: Images\WN25UJW\WN25UJW_4.jpg
Watermark added to Images\WN25UJW\WN25UJW_5.jpg
Downloaded: Images\WN25UJW\WN25UJW_5.jpg
Watermark added to Images\WN25UJW\WN25UJW_6.jpg
Downloaded: Images\WN25UJW\WN25UJW_6.jpg
Watermark added to Images\WN25UJW\WN25UJW_7.jpg
Downloaded: Images\WN25UJW\WN25UJW_7.jpg
Watermark added to Images\WN25UJW\WN25UJW_8.jpg
Downloaded: Images\WN25UJW\WN25UJW_8.jpg
Watermark added to Images\WN25UJW\WN25UJW_9.jpg
Downloaded: Images\WN25UJW\WN25UJW_9.jpg
Watermark added to Images\WN25UJW\WN25UJW_10.jpg
Downloaded: Images\WN25UJW\WN25UJW_10.jpg
Watermark added to Images\WN25UJ